In [134]:
import math as ma
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.initializers import glorot_uniform
from keras.layers import Dense, LSTM, Dropout, TimeDistributed, Activation, Input, concatenate
from keras.callbacks import EarlyStopping
import datetime as dt
import matplotlib.pyplot as plt
import tensorflow as tf

from keras import optimizers
from keras.callbacks import History

In [197]:
def readData(data_path):
  return pd.read_csv(data_path)

def generateData(df):
  anios = df.iloc[:, 0].tolist()
  temperaturas = df.iloc[:, 1].tolist()
  data=df.iloc[:, 2:]
  anios=np.array(anios)
  temperaturas=np.array(temperaturas)
  data=np.array(data)
  return anios,temperaturas,data

def normData(data):
  scaler = MinMaxScaler()
  data = scaler.fit_transform(data)
  return data

def modelo(data):
  model = Sequential()
  model.add(LSTM(128, return_sequences= True, input_shape=(290, data.shape[2])))
  model.add(Dropout(0.1))
  model.add(LSTM(128, return_sequences= False))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation= 'linear'))
  model.compile(optimizer=optimizers.Adam(), loss='mse')
  return model

# def entrenar_modelo(modelo,epocas,train_data,targets,testx=0,testy=0):
#   scores=list()
#   histories=list()
#   history=modelo.fit(train_data, targets, epochs=epocas, batch_size=32, verbose=1)
#   #_, acc = model.evaluate(testx, testy, verbose=1)
#   #print('> %.3f' % (acc * 100.0))
#   #scores.append(acc)
#   #histories.append(history)
#   #return scores, histories, model

def entrenar_modelo(modelo, epocas, train_data, train_targets, val_data, val_targets):

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = modelo.fit(
        train_data,
        train_targets,
        epochs=epocas,
        batch_size=16,
        validation_data=(val_data, val_targets),
        verbose=1,
        shuffle=False,  # No shuffle para series temporales
        callbacks=[early_stopping]
      )

    return history

def graphYVT(x,y,type):
  plt.plot(x, y, label=type, color=np.random.rand(3,), linestyle='-', linewidth=2)
  plt.title('Anio VS. Temp')
  plt.xlabel('Anio')
  plt.legend()
  plt.ylabel('Temperatura')

def generateTestData(anio, temp, trainData):
  test_mat=np.insert(trainData,0,anio,axis=1)
  test_mat=np.insert(test_mat,1,temp,axis=1)

  medias_por_anio = []
  anios_unicos = np.unique(test_mat[:, 0])
  # Iterar sobre cada año único
  for anio in anios_unicos:
    filas_anio = test_mat[test_mat[:, 0] == anio]
    promedio = np.mean(filas_anio, axis=0)
    medias_por_anio.append(promedio)

  return np.array(medias_por_anio)


In [198]:
def graphInputData(data):
  # Calcular los promedios de cada variable agrupados por año
  yearly_means = data.groupby('Year').mean()

  # Configurar el tamaño de los gráficos
  num_variables = len(yearly_means.columns)
  fig, axes = plt.subplots(nrows=num_variables // 2 + num_variables % 2, ncols=2, figsize=(15, num_variables * 2))
  axes = axes.flatten()

  # Generar un gráfico para cada variable
  for idx, column in enumerate(yearly_means.columns):
      axes[idx].plot(yearly_means.index, yearly_means[column], label=column, color='b')
      axes[idx].set_title(f'Average {column} by Year')
      axes[idx].set_xlabel('Year')
      axes[idx].set_ylabel(column)
      axes[idx].grid()
      axes[idx].legend()


  # Obtener estadísticas descriptivas. Esto ayuda a identificar valores atípicos o rangos inusuales en las variables.
  stats = data.describe()
  print("DATOS ESTADISTICOS DE CADA COLUMNA DE LA TABLA DE DATOS")
  print(stats)
  print()
  print(100*'-')
  # Ajustar el diseño para evitar solapamientos
  plt.tight_layout()
  plt.show()


def get_cleared_data(data):
  # Verificar la cantidad de registros por año. Esto asegura que haya uniformidad en el número de registros por año.
  year_distribution = data['Year'].value_counts().sort_index()
  print('CANTIDAD DE REGISTROS POR AÑO')
  print(year_distribution)
  print()
  print(100*'-')

  # Calcular correlación con T2M_1. Identifica las variables que están más relacionadas con la temperatura
  correlation_with_temp = data.corr()['T2M_1'].sort_values(ascending=False)

  # Filtrar las columnas con correlación mayor al 50%
  columns_to_keep = correlation_with_temp[correlation_with_temp > 0.3].index

  # Crear un nuevo DataFrame solo con esas columnas
  filtered_data = data[columns_to_keep]

  #Agregar columna year al principio
  year_column = data.pop('Year')
  filtered_data.insert(0, 'Year', year_column)

  # Mostrar el resultado
  print('COLUMNAS FILTRADAS EN BASE A LA CORRELACION CON LA TEMPERATURA')
  print(filtered_data)
  print()
  print(100*'-')

  #Ordenar por año
  filtered_data = filtered_data.sort_values(by='Year')

  return filtered_data

In [207]:
def predict_future_temperatures(model, last_data, num_predictions):
  """
  Predice las temperaturas futuras utilizando una ventana deslizante.

  Args:
    model: El modelo LSTM entrenado.
    last_data: Los últimos datos utilizados para iniciar la predicción.
    num_predictions: El número de predicciones futuras a generar.

  Returns:
    Una lista de temperaturas predichas.
  """
  predictions = []
  current_data = last_data
  for _ in range(num_predictions):
    # Predecir el siguiente valor
    next_prediction = model.predict(current_data)

    # Agregar la predicción a la lista
    predictions.append(next_prediction[0, 0]) #Tomamos el primer valor

    # Desplazar la ventana de predicción
    current_data = np.roll(current_data, -1, axis=1)
    current_data[0, -1] = next_prediction

  return predictions


In [189]:
data = readData('./trainData.csv')
cleared_data = get_cleared_data(data);
graphInputData(cleared_data);

CANTIDAD DE REGISTROS POR AÑO
Year
1984.0    290
1985.0    290
1986.0    290
1987.0    290
1988.0    290
1989.0    290
1990.0    290
1991.0    290
1992.0    290
1993.0    290
1994.0    290
1995.0    290
1996.0    290
1997.0    290
1998.0    290
1999.0    290
2000.0    290
2001.0    290
2002.0    290
2003.0    290
2004.0    290
2005.0    290
2006.0    290
2007.0    290
2008.0    290
2009.0    290
2010.0    290
2011.0    290
2012.0    290
2013.0    290
2014.0    290
2015.0    290
2016.0    290
2017.0    290
2018.0    290
2019.0    290
2020.0    290
2021.0    290
2022.0    290
Name: count, dtype: int64

----------------------------------------------------------------------------------------------------
COLUMNAS FILTRADAS EN BASE A LA CORRELACION CON LA TEMPERATURA
         Year      T2M_1     VAP_1     VPD_1  CDD18_3_1  ALLSKY_SFC_LW_DWN_1  \
0      1984.0  24.520645  3.119521  3.118364   5.924194           389.741935   
1      1984.0  23.613871  2.943558  2.942219   5.133548           40

In [200]:
anios,temperaturas,trainData=generateData(cleared_data)

# Define los puntos de división
train_size = int(len(trainData) * 0.6)  # 60% para entrenamiento
val_size = int(len(trainData) * 0.2)    # 20% para validacion y 20% para testing

#Divide los datos
train_data = trainData[:train_size]
val_data = trainData[train_size:train_size + val_size]
test_data = trainData[train_size + val_size:]

anios_train = anios[:train_size]  #Del 1984 hasta algunos del 2007
anios_val = anios[train_size:train_size + val_size] #Desde el 2007 hasta algunos de 2015
anios_test = anios[train_size + val_size:] # Desde 2015 hasta el 2022

train_targets = temperaturas[:train_size]
val_targets = temperaturas[train_size:train_size + val_size]
test_targets = temperaturas[train_size + val_size:]

train_data = normData(train_data)
val_data = normData(val_data)
test_data = normData(test_data)

train_data = np.expand_dims(train_data, axis=1)
val_data = np.expand_dims(val_data, axis=1)
test_data = np.expand_dims(test_data, axis=1)

model=modelo(train_data)
model.summary()
history = entrenar_modelo(modelo=model,
                epocas=100,
                train_data=train_data,
                train_targets=train_targets,
                val_data=val_data,
                val_targets=val_targets)
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_34 (LSTM)                       │ (None, 290, 128)            │          69,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 290, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_35 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 201,345 (786.50 KB)

 Trainable params: 201,345 (786.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_34 (LSTM)                       │ (None, 290, 128)            │          69,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 290, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_35 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 604,037 (2.30 MB)

 Trainable params: 201,345 (786.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 402,692 (1.54 MB)

In [ ]:
# Evaluar el modelo en los datos de validación
loss = model.evaluate(test_data, test_targets, verbose=1)
print("Pérdida en los datos de validación:", loss)

# Predecir temperaturas
predicted_temperatures = model.predict(test_data)

# Crear un DataFrame con los años y las temperaturas
df_test = pd.DataFrame({'Year': anios_test, 'Real': test_targets, 'Predicción': predicted_temperatures.flatten()})

# Filtrar los datos para los años entre 2017 y 2022
df_test_filtered = df_test.loc[(df_test['Year'] >= 2017) & (df_test['Year'] <= 2022)]

# Agrupar por año y calcular la media
yearly_means = df_test_filtered.groupby('Year').mean()

# Obtener los años únicos
years = yearly_means.index.values

# Obtener las temperaturas reales y predichas promedio por año
real_temperatures = yearly_means['Real'].values
predicted_temperatures = yearly_means['Predicción'].values

print(years)
print(real_temperatures)
print(predicted_temperatures)

plt.plot(years, real_temperatures, label='Real', color='blue')
plt.plot(years, predicted_temperatures, label='Predicción', color='red')
plt.title('Comparación de temperaturas reales y predichas (promedio anual)')
plt.xlabel('Año')
plt.ylabel('Temperatura')
plt.legend()
plt.show()


# print(test_data[-870:])
# print(test_targets[-870:])
# print(anios_test[-870:])

# Suponiendo que tu modelo LSTM usa una secuencia de 3 años
last_data = test_data[-870:].reshape(1, 870, test_data.shape[2])

# Predecir las temperaturas para los próximos 5 años
future_predictions = predict_future_temperatures(model, last_data, 290)
average_prediction = np.mean(future_predictions)

# Imprimir las predicciones en la escala original
print(average_prediction)

In [ ]:
# anios,temperaturas,trainData=generateData(cleared_data)

# test_data = generateTestData(anios,temperaturas,trainData)

# trainData=normData(trainData)

# trainData=np.expand_dims(trainData,axis=1)

# model=modelo(trainData)
# model.summary()
# entrenar_modelo(modelo=model,
#                 epocas=80,
#                 train_data=trainData,
#                 targets=temperaturas,
#                 testx=0,
#                 testy=0)
# model.summary()

# ## Predecir temperaturas

# predicted_temperatures = PredictTemperatures(model, trainData)

##esto del preducit esta mal, tenemos que usar una secuencia de un anio especifico para predecir la que sigue
##https://chatgpt.com/c/67452d60-9a14-8004-9adf-85575afb697d

In [ ]:
#aux1,testTarget,testData=generateData('./testData.csv')
#testData=normData(testData)
#testData=np.expand_dims(testData,axis=1)
#last_data= testData[-1:].reshape(1,1,28)
#last_data= trainData[-30:].reshape(1, 30, 28)

#prediccion= model.predict(last_data)
#print(prediccion)
##graphYVT(anios,temperaturas,'train')
##graphYVT(aux1,testTarget,'test')
##graphYVT(aux1,prediccion,'predicction')
##plt.show()
##rmse = np.sqrt(np.mean((prediccion - testTarget)**2))
##print(rmse)

FileNotFoundError: [Errno 2] No such file or directory: './testData.csv'

In [ ]:
()